In [8]:
import numpy as np

# Load the data from the provided .npz file
data = np.load('output/2D/2023cvpr_keypoints.npz')
print(data.files)  # Inspect the keys in the loaded data


['reconstruction']


In [9]:
# Inspect the shape and contents of the 'reconstruction' dataset
reconstruction_data = data['reconstruction']
reconstruction_data.shape


(1, 183, 17, 2)

In [10]:
# Inspect the shape and some initial values of the 'reconstruction' dataset
reconstruction_data = data['reconstruction']
shape = reconstruction_data.shape
sample_values = reconstruction_data[:2]  # Display the first two sets of values for inspection

shape, sample_values

((1, 183, 17, 2),
 array([[[[1046.315  ,  393.3288 ],
          [1015.0746 ,  391.97052],
          [1020.5077 ,  522.36505],
          ...,
          [ 993.34216,  234.41046],
          [ 985.1925 ,  315.90704],
          [ 979.7594 ,  389.25397]],
 
         [[1046.235  ,  394.36206],
          [1015.1766 ,  393.01172],
          [1020.578  ,  522.64685],
          ...,
          [ 993.5707 ,  233.66849],
          [ 982.76776,  317.3912 ],
          [ 980.067  ,  387.61026]],
 
         [[1045.95   ,  393.89813],
          [1014.98083,  392.55167],
          [1020.36676,  521.81415],
          ...,
          [ 993.4371 ,  233.6665 ],
          [ 985.35815,  317.14853],
          [ 979.9722 ,  387.1657 ]],
 
         ...,
 
         [[1078.7318 ,  763.52026],
          [1042.1611 ,  754.03894],
          [ 966.3107 ,  913.8665 ],
          ...,
          [ 969.0197 ,  913.8665 ],
          [ 958.1839 ,  838.0161 ],
          [ 966.3107 ,  819.0535 ]],
 
         [[1067.8782 ,  777.17

In [11]:
import plotly.graph_objs as go
from ipywidgets import interact

# 骨骼连接数据
connections = [
    [0, 1], [1, 2], [2, 3], [0, 4], [4, 5],
    [5, 6], [0, 7], [7, 8], [8, 9], [9, 10],
    [8, 11], [11, 12], [12, 13], [8, 14], [14, 15], [15, 16]
]
# 点的标签


In [12]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import widgets

In [13]:
def calculate_angle_between_vectors(v1, v2):
    """Calculate the angle in degrees between two vectors."""
    unit_vector_1 = v1 / np.linalg.norm(v1)
    unit_vector_2 = v2 / np.linalg.norm(v2)
    dot_product = np.dot(unit_vector_1, unit_vector_2)
    angle = np.arccos(dot_product)
    return np.degrees(angle)

# 假设有方法获取关节位置
def get_joint_position(frame_data, index):
    return frame_data[index]

# 添加角度显示到图形中
def add_angle_display(fig, pos1, pos2, pos3, label):
    vector1 = pos2 - pos1
    vector2 = pos2 - pos3
    angle = calculate_angle_between_vectors(vector1, vector2)
    angle_text = f"{label} Angle: {angle:.2f}°"
    fig.add_trace(go.Scatter3d(
        x=[pos2[0]],  
        y=[pos2[2]],  # 适应您的轴设置
        z=[-pos2[1]],
        text=[angle_text],
        mode='text',
        textposition="top center"
    ))

# 添加角度显示到图形中
def add_angle_displayV2(fig, pos1, pos2, vector2, label):
    vector1 = pos2 - pos1
    vector2 = vector2
    angle = calculate_angle_between_vectors(vector1, vector2)
    angle_text = f"{label} Angle: {angle:.2f}°"
    fig.add_trace(go.Scatter3d(
        x=[pos2[0]],  
        y=[pos2[2]],  # 适应您的轴设置
        z=[-pos2[1]],
        text=[angle_text],
        mode='text',
        textposition="top center"
    ))

In [16]:
# 初始化FigureWidget
fig = go.FigureWidget()

# 设置布局
fig.update_layout(scene=dict(
                    xaxis=dict(title='X'),
                    yaxis=dict(title='Y'),  # 确保y轴垂直方向
                    zaxis=dict(title='Z')
                ))


labels = ['Hips', 'LeftThight', 'RightShin', 'RightFoot', 'RightThigh', 'LeftShin', 'LeftFoot', 'Spine', 'Neck', 'UpperNeck', 'TopHead', 'LeftShoulder', 'LeftArm', 'LeftHand', 'RightShoulder', 'RightArm', 'RightHand']

# 为FigureWidget添加初始帧的数据
def init_figure():
    frame_data = reconstruction_data[0]
    print(frame_data)
    for connection in connections:
        x = [frame_data[connection[0], 0], frame_data[connection[1], 0]]
        y = [frame_data[connection[0], 2], frame_data[connection[1], 2]]  # 反转原z轴的值
        z = [-frame_data[connection[0], 1], -frame_data[connection[1], 1]]  # 使用原y轴的值
        fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='lines', line=dict(color='blue'), showlegend=False))
    # fig.add_trace(go.Scatter3d(x=frame_data[:, 0], y=frame_data[:, 2], z=-frame_data[:, 1], mode='markers', marker=dict(size=5, color='red')))

    # # 示例：计算并显示大腿和小腿的角度
    # right_thigh_pos = get_joint_position(frame_data, labels.index('RightThigh'))
    # right_shin_pos = get_joint_position(frame_data, labels.index('RightShin'))
    # right_foot_pos = get_joint_position(frame_data, labels.index('RightFoot'))
    
    add_angle_display(fig, right_thigh_pos, right_shin_pos, right_foot_pos, "Right Leg")


    # 示例：计算并显示大腿和小腿的角度
    Hips_pos = get_joint_position(frame_data, labels.index('Hips'))
    Head_pos = get_joint_position(frame_data, labels.index('TopHead'))
    vertical_vector =  np.array([0, 0, 1])  # 取默认的垂直向量 
    
    add_angle_displayV2(fig, Head_pos, Hips_pos, vertical_vector, "Body")

    # 对于每个标签添加一个点到图例
    for i, label in enumerate(labels):
        fig.add_trace(go.Scatter3d(
            x=[frame_data[i, 0]],
            y=[frame_data[i, 2]],
            z=[-frame_data[i, 1]],
            mode='markers',
            marker=dict(size=5, color='red'),  # 指定颜色为红色            
            name=label,
        ))
init_figure()

# 创建滑动条
slider = widgets.IntSlider(min=0, max=reconstruction_data.shape[0] - 1, step=1, value=0)

# 滑动条的回调函数
def update_frame(change):
    frame_data = reconstruction_data[change.new]
    with fig.batch_update():
        fig.data = []
        for connection in connections:
            x = [frame_data[connection[0], 0], frame_data[connection[1], 0]]
            y = [frame_data[connection[0], 2], frame_data[connection[1], 2]]  # 反转原z轴的值
            z = [-frame_data[connection[0], 1], -frame_data[connection[1], 1]]  # 使用原y轴的值
            fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='lines', line=dict(color='blue'), showlegend=False))
        # fig.add_trace(go.Scatter3d(x=frame_data[:, 0], y=frame_data[:, 2], z=-frame_data[:, 1], mode='markers', marker=dict(size=5, color='red')))

    # 示例：计算并显示大腿和小腿的角度
    right_thigh_pos = get_joint_position(frame_data, labels.index('RightThigh'))
    right_shin_pos = get_joint_position(frame_data, labels.index('RightShin'))
    right_foot_pos = get_joint_position(frame_data, labels.index('RightFoot'))
    
    add_angle_display(fig, right_thigh_pos, right_shin_pos, right_foot_pos, "Right Leg")

    # 示例：计算并显示大腿和小腿的角度
    Hips_pos = get_joint_position(frame_data, labels.index('Hips'))
    Head_pos = get_joint_position(frame_data, labels.index('Neck'))
    vertical_vector =  np.array([0, 1, 0])  # 取默认的垂直向量 
    
    add_angle_displayV2(fig, Head_pos, Hips_pos, vertical_vector, "Body")

    # 对于每个标签添加一个点到图例
    for i, label in enumerate(labels):
        fig.add_trace(go.Scatter3d(
            x=[frame_data[i, 0]],
            y=[frame_data[i, 2]],
            z=[-frame_data[i, 1]],
            mode='markers',
            marker=dict(size=5, color='red'),  # 指定颜色为红色            
            name=label,
        ))
slider.observe(update_frame, names='value')

display(slider, fig)

[[[1046.315    393.3288 ]
  [1015.0746   391.97052]
  [1020.5077   522.36505]
  ...
  [ 993.34216  234.41046]
  [ 985.1925   315.90704]
  [ 979.7594   389.25397]]

 [[1046.235    394.36206]
  [1015.1766   393.01172]
  [1020.578    522.64685]
  ...
  [ 993.5707   233.66849]
  [ 982.76776  317.3912 ]
  [ 980.067    387.61026]]

 [[1045.95     393.89813]
  [1014.98083  392.55167]
  [1020.36676  521.81415]
  ...
  [ 993.4371   233.6665 ]
  [ 985.35815  317.14853]
  [ 979.9722   387.1657 ]]

 ...

 [[1078.7318   763.52026]
  [1042.1611   754.03894]
  [ 966.3107   913.8665 ]
  ...
  [ 969.0197   913.8665 ]
  [ 958.1839   838.0161 ]
  [ 966.3107   819.0535 ]]

 [[1067.8782   777.1787 ]
  [1020.5081   781.239  ]
  [ 969.0778   913.875  ]
  ...
  [1128.7823   924.70245]
  [ 955.5435   838.083  ]
  [ 966.371    816.42816]]

 [[1090.8533   773.2233 ]
  [1139.5455   773.2233 ]
  [1174.7122   819.2104 ]
  ...
  [1150.366    922.005  ]
  [1180.1223   843.55646]
  [1174.7122   816.50525]]]


NameError: name 'right_thigh_pos' is not defined